### Start by installing CMAKE
https://cmake.org/download/

# Facial Recognition

In [2]:
!pip install opencv-python

  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)


In [20]:
import cv2

# Specify the full path to the haarcascade_frontalface_default.xml file
cascade_path = 'haarcascade_frontalface_default.xml'

# Load the pre-trained face detection classifier
face_cascade = cv2.CascadeClassifier(cascade_path)

# Start capturing video from the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Facial Recognition', frame)

    # Check if the user pressed the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
video_capture.release()
cv2.destroyAllWindows()


# Liveness Detection

In [1]:
!pip install opencv-python mediapipe

  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     ------------------------------------- 162.8/162.8 kB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.4/50.8 MB 12.6 MB/s eta 0:00:05
    --------------------------------------- 0.8/50.8 MB 13.4 MB/s eta 0:00:04
    --------------------------------------- 0.9/50.8 MB 9.8 MB/s eta 0:00:06
   - -------------------------------------- 1.5/50.8 MB 11.8 MB/s eta 0:00:05
   - -------------------------------------- 2.2/50.8 MB 12.6 MB/s eta 0:00:04
   -- ------------------------------------- 2.8/50.8 MB 14.0 MB/s eta 0:00:04
   -- ------------

In [19]:
# TRY 7

import cv2
import mediapipe as mp

# Initialize mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image and get the hand landmarks
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw colored squares around landmarks for visualization
            for lm in hand_landmarks.landmark:
                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.rectangle(frame, (cx - 5, cy - 5), (cx + 5, cy + 5), (255, 0, 0), -1)

            # Check if the detected hand is the left hand
            if hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x < hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x:
                wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
                middle_finger_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                if wrist_y > middle_finger_tip_y:
                    cv2.putText(frame, 'Left hand lifted!', (frame.shape[1] - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, 'Left hand down', (frame.shape[1] - 300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
                middle_finger_tip_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                if wrist_y > middle_finger_tip_y:
                    cv2.putText(frame, 'Right hand lifted!', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, 'Right hand down', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Liveness Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()